Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [14]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf
import sklearn
import warnings
warnings.filterwarnings("ignore")

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [15]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'TSLA'

In [16]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, start="2019-01-01", end="2021-01-01")
df = df[df.columns[0:4]]
df = df[['Open', 'High', 'Low', 'Close']]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2018-12-31,67.557999,67.842003,65.052002,66.559998
2019-01-02,61.220001,63.026001,59.759998,62.023998
2019-01-03,61.400002,61.880001,59.476002,60.071999
2019-01-04,61.200001,63.599998,60.546001,63.537998
2019-01-07,64.344002,67.348000,63.549999,66.991997
...,...,...,...,...
2020-12-24,642.989990,666.090027,641.000000,661.770020
2020-12-28,674.510010,681.400024,660.799988,663.690002
2020-12-29,661.000000,669.900024,655.000000,665.989990


In [17]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window = 15).mean()
df['Open_1_Change'] = df['Open'] .diff()
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2019-01-22,60.964001,61.599998,59.099998,59.784000,65.805466,-3.635998
2019-01-23,58.500000,58.900002,56.338001,57.518002,65.202666,-2.464001
2019-01-24,56.605999,58.736000,55.855999,58.301998,64.954533,-1.894001
2019-01-25,58.877998,59.703999,57.910000,59.408001,64.910267,2.271999
2019-01-28,58.582001,59.492001,57.549999,59.276001,64.626133,-0.295998
...,...,...,...,...,...,...
2020-12-24,642.989990,666.090027,641.000000,661.770020,638.461336,10.789978
2020-12-28,674.510010,681.400024,660.799988,663.690002,642.771338,31.520020
2020-12-29,661.000000,669.900024,655.000000,665.989990,644.386670,-13.510010


In [18]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df.Close.shift(-1)-df['Close'])>=0,1,-1)

In [19]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train = X[:index]
X_test = X[index:]
y_train = y[:index]
y_test = y[index:]

In [20]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)

In [21]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[-0.03942939995611678]
1,High,[0.01949077524517122]
2,Low,[0.08479284522373805]
3,Close,[-0.059249361620604575]
4,Close_15_Rolling,[-0.0029236753975663255]
5,Open_1_Change,[0.029595292538085915]


In [22]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [23]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_pred=predictions,y_true=y_test)

array([[16, 45],
       [24, 63]], dtype=int64)

In [24]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test,y_test)

0.5337837837837838

In [25]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val= cross_val_score(LogisticRegression(), X, y, cv=5)
cross_val

array([0.55555556, 0.53535354, 0.48979592, 0.53061224, 0.56122449])